<a href="https://colab.research.google.com/github/amoutafian/DataScienceGuidedCapstone/blob/master/medicare_inpatient_charges_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.ticker as tick
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
from google.colab import auth
auth.authenticate_user()
project_id = 'medicare-charges-project'

In [ ]:
Describe dataset

SyntaxError: ignored

In [ ]:
how many rows, years info, how data organized (5 years individual datasets 2011-2015)

Used Union to combine year-specific datasets into one main dataset comprising all data from all years (2011-1015)


In [ ]:
source_dataset = pd.io.gbq.read_gbq('''
  WITH source_table AS (
  SELECT *, 2011 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2011`
UNION ALL
SELECT *, 2012 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2012`
UNION ALL
SELECT *, 2013 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2013`
UNION ALL
SELECT *, 2014 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2014`
UNION ALL
SELECT *, 2015 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2015`),

yearly_count AS (SELECT COUNT(*) AS count_year, year
FROM source_table
GROUP BY year)

SELECT * FROM yearly_count
ORDER BY year;

''', project_id=project_id)

In [ ]:
source_dataset

This query counts providers per state for a specific DRG code.

In [ ]:
df2 = pd.io.gbq.read_gbq('''
WITH source_table AS (
SELECT *, 2011 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2011`
UNION ALL
SELECT *, 2012 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2012`
UNION ALL
SELECT *, 2013 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2013`
UNION ALL
SELECT *, 2014 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2014`
UNION ALL
SELECT *, 2015 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2015`)
select
  sum(total_discharges) as total_discharges,
  provider_state,
  drg_definition,
  count(*) as total_providers
from
  source_table
  where
  drg_definition = '244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O CC/MCC'
  group by 2,3
  order by provider_state;

''', project_id=project_id)

In [ ]:
df2

Next Steps! Define the source table and save for repeated access.

Null checks and data set row counts, replicate previous projects for data info.

In [ ]:
df3 = pd.io.gbq.read_gbq('''
WITH source_table AS (
SELECT *, 2011 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2011`
UNION ALL
SELECT *, 2012 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2012`
UNION ALL
SELECT *, 2013 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2013`
UNION ALL
SELECT *, 2014 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2014`
UNION ALL
SELECT *, 2015 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2015`)
SELECT
  drg_definition,
  provider_state,
  sum(total_discharges) AS total_discharges,
  count(*) AS total_providers
FROM
  source_table
  GROUP BY 1,2
  ORDER BY 1,2
''', project_id=project_id)

df3

,drg_definition,provider_state,total_discharges,total_providers
0,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,AL,24,2
1,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,AZ,48,2
2,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CA,280,12
3,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CO,11,1
4,001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...,CT,58,4
...,...,...,...,...
17021,988 - NON-EXTENSIVE O.R. PROC UNRELATED TO PRI...,WI,54,4
17022,988 - NON-EXTENSIVE O.R. PROC UNRELATED TO PRI...,WV,39,3
17023,989 - NON-EXTENSIVE O.R. PROC UNRELATED TO PRI...,KS,26,2
17024,989 - NON-EXTENSIVE O.R. PROC UNRELATED TO PRI...,KY,13,1


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3903 entries, 0 to 3902
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   drg_definition  3903 non-null   object
 1   provider_id     3903 non-null   object
 2   provider_name   3903 non-null   object
 3   f0_             3903 non-null   Int64 
dtypes: Int64(1), object(3)
memory usage: 125.9+ KB


In [ ]:
df_discharge_count = pd.io.gbq.read_gbq('''
WITH source_table AS (
SELECT *, 2011 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2011`
UNION ALL
SELECT *, 2012 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2012`
UNION ALL
SELECT *, 2013 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2013`
UNION ALL
SELECT *, 2014 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2014`
UNION ALL
SELECT *, 2015 AS year
FROM `bigquery-public-data.cms_medicare.inpatient_charges_2015`)
select
  drg_definition,
  provider_id,
  provider_name,
  sum(total_discharges)
from source_table
where
  drg_definition like '%PACEMAKER%'
  --'244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O CC/MCC' or drg_definition = ''
group by 1,2,3
order by 4 desc
''', project_id=project_id)

In [ ]:
df_discharge_count

,drg_definition,provider_id,provider_name,f0_
0,244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O ...,330182,"ST FRANCIS HOSPITAL, ROSLYN",552
1,243 - PERMANENT CARDIAC PACEMAKER IMPLANT W CC,100007,FLORIDA HOSPITAL,518
2,244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O ...,330106,NORTH SHORE UNIVERSITY HOSPITAL,457
3,243 - PERMANENT CARDIAC PACEMAKER IMPLANT W CC,330182,"ST FRANCIS HOSPITAL, ROSLYN",450
4,244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O ...,330101,NEW YORK-PRESBYTERIAN HOSPITAL,441
...,...,...,...,...
3898,244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O ...,50359,TULARE REGIONAL MEDICAL CENTER,11
3899,243 - PERMANENT CARDIAC PACEMAKER IMPLANT W CC,50688,SAINT LOUISE REGIONAL HOSPITAL,11
3900,244 - PERMANENT CARDIAC PACEMAKER IMPLANT W/O ...,50254,MARSHALL MEDICAL CENTER (1-RH),11
3901,243 - PERMANENT CARDIAC PACEMAKER IMPLANT W CC,120002,MAUI MEMORIAL MEDICAL CENTER,11
